In [2]:
!pip -q install pyspark transformers tokenizers torch torchvision sentencepiece protobuf

     |████████████████████████████████| 310.8 MB 10 kB/s s eta 0:00:01    |████████▎                       | 80.2 MB 22.0 MB/s eta 0:00:11     |██████████▎                     | 100.1 MB 30.7 MB/s eta 0:00:07     |███████████▋                    | 113.0 MB 30.7 MB/s eta 0:00:07     |███████████████████████████▉    | 270.1 MB 283 kB/s eta 0:02:24     |██████████████████████████████  | 290.5 MB 283 kB/s eta 0:01:12     |███████████████████████████████ | 301.1 MB 19.0 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 6.8 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285413 sha256=ce39582314b3d27fa5b56a7acbc2b9f59f4b40c88eaf85a8eac436ac8eb46b44
  Stored in directory: /root/.cache/pip/wheels/07/9f/04/fc2c478c8c87334f0ff48e86fa7d8c2a814f9df5dc21d79b7e
Successfully built pyspark
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [13]:
!apt-get update && apt-get install -y default-jre

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://security.debian.org/debian-security buster/updates InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Hit:4 https://deb.nodesource.com/node_12.x buster InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  adwaita-icon-theme at-spi2-core ca-certificates-java dbus dbus-user-session
  dconf-gsettings-backend dconf-service default-jre-headless dmsetup
  fonts-dejavu-extra glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas gtk-update-icon-cache
  java-common libargon2-1 libasound2 libasound2-data libatk-bridge2.0-0
  libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0 libatk1.0-data
  libatspi2.0-0 libcolord2 libcryptsetup12 libdbus-1-3 libdconf1
  libdevmapper1.02.1 libepoxy0 libgail-common libgail18 libgif7 libgtk-3-0
  libgtk

In [14]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Debian-1deb10u1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Debian-1deb10u1, mixed mode, sharing)


In [60]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import count, col, lower
from transformers import pipeline

/usr/local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [8]:
# Define your ClickHouse JDBC URL
clickhouse_url = "jdbc:clickhouse://clickhouse:8123"

In [3]:
# Create a Spark configuration
conf = SparkConf().setAppName("ClickhouseETL")

# Add the clickhouse-jdbc JAR file to the Spark configuration
conf.set("spark.jars", "/notebooks/clickhouse-jdbc-0.2.4.jar")

# Create a SparkContext
sc = SparkContext(conf=conf, master="local[*]")

# Create a Spark session
spark = SparkSession(sc)

sc = spark.sparkContext

In [9]:
# Define your connection properties
connection_properties = {
    "user": "clickhouse-user",
    "password": "secret",
    "driver": "ru.yandex.clickhouse.ClickHouseDriver"
}

In [10]:
# Define the table you want to read from
table_name = "messages"

In [11]:
# Read data from ClickHouse
clickhouse_data = spark.read.jdbc(
    url=clickhouse_url,
    table=table_name,
    properties=connection_properties
)

In [12]:
# You can now perform operations on the clickhouse_data DataFrame
clickhouse_data.show()

+--------------------+-----------+--------------------+-------------+-------------------+-----------+-----------+------------+--------+--------------------+--------------------+--------------------+--------+-----------+
|                  id|       type|             message|    timestamp|           datetime|elapsedTime|amountValue|amountString|currency|         author_name|    author_channelId|   author_channelUrl|video_id|stream_name|
+--------------------+-----------+--------------------+-------------+-------------------+-----------+-----------+------------+--------+--------------------+--------------------+--------------------+--------+-----------+
|ChwKGkNPenhuTlcwa...|textMessage|COBRAR PLANES?? M...|1693880519477|2023-09-05 02:21:59|           |        0.0|            |        |     joaquin salagre|UCQpDqT6hO939xyEF...|http://www.youtub...|        |           |
|ChwKGkNJcmEwOWEwa...|textMessage|"vamos"? si estás...|1693880522467|2023-09-05 02:22:02|           |        0.0|       

In [23]:
clickhouse_data.groupBy("author_name").agg(count("*").alias("count")).orderBy(col("count").desc()).show()

+--------------------+-----+
|         author_name|count|
+--------------------+-----+
|               Yahoo|   16|
|Braian Ariel Barraza|   15|
|        Simpeador77 |   11|
|              Future|    9|
|     Hugo Oneto Sosa|    9|
|          framebyfer|    8|
|       TORRIJO PABLO|    6|
|           HELL_FIRE|    5|
|    Alejandro Ungaro|    5|
|           Ulises Gs|    4|
|     joaquin salagre|    4|
|        Alancay Pity|    3|
|Jeremias Ezequiel...|    2|
|    Federico Frehley|    2|
|          Ema Moreno|    2|
|       Gustavo Tello|    2|
|        gonzalo pepe|    1|
|                90,s|    1|
|     Fer Clonazepum!|    1|
|              Alan89|    1|
+--------------------+-----+
only showing top 20 rows



In [24]:
jxc_words = ["pato", "patricia", "bullrich", "jxc", "macri", "larreta", "pelado", "jorge"]
lla_words = ["libertad", "milei", "piparo", "villarruel", "javier", "leon"]
k_words = ["cristina", "kirchner", "fernandez", "massa", "sergio", "tomas", "kicillof", "alberto"]

In [36]:
interesting_data = clickhouse_data.withColumn("is_milei", lower(col("message")).rlike("|".join(lla_words))) \
    .withColumn("is_massa", lower(col("message")).rlike("|".join(k_words))) \
    .withColumn("is_bullrich", lower(col("message")).rlike("|".join(jxc_words))) \
    .where(col("is_massa") | col("is_bullrich") | col("is_milei"))

In [37]:
interesting_data.show()

+--------------------+-----------+--------------------+-------------+-------------------+-----------+-----------+------------+--------+----------------+--------------------+--------------------+--------+-----------+--------+--------+-----------+
|                  id|       type|             message|    timestamp|           datetime|elapsedTime|amountValue|amountString|currency|     author_name|    author_channelId|   author_channelUrl|video_id|stream_name|is_milei|is_massa|is_bullrich|
+--------------------+-----------+--------------------+-------------+-------------------+-----------+-----------+------------+--------+----------------+--------------------+--------------------+--------+-----------+--------+--------+-----------+
|ChwKGkNKQ29qdXEwa...|textMessage|VILLARRUEL QUE MU...|1693880563273|2023-09-05 02:22:43|           |        0.0|            |        | Hugo Oneto Sosa|UCnI2pK9hPjuPWUED...|http://www.youtub...|        |           |    true|   false|      false|
|ChwKGkNQRDlzZk8

In [61]:
def analyze_sentiment_partition(iterator):
    # Initialize the Hugging Face sentiment analysis pipeline
    # TODO uncomment this: 
    pipe = pipeline("text-classification", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")
    
    # Process each row in the partition
    for row in iterator:
        text = row["message"]
        sentiment_result = pipe(text)
        # Create a dictionary from the original row
        row_dict = row.asDict()
        
        # Add the sentiment result as a new key-value pair in the dictionary
        row_dict['sentiment_label'] = sentiment_result[0]["label"]
        row_dict['sentiment_score'] = sentiment_result[0]["score"]
        
        # Convert the dictionary back to a Row
        updated_row = Row(**row_dict)
        
        yield updated_row

In [ ]:
result_rdd = interesting_data.repartition(1).rdd.mapPartitions(analyze_sentiment_partition)
result_df = result_rdd.toDF()

In [ ]:
result_df.show()